In [3]:
import sys
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport
from ipywidgets import widgets

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df_test = test.copy()
df_train = train.copy()

submission = pd.read_csv('sample_submission.csv')
price = pd.read_csv('price.csv')

tasit = pd.read_csv('tasitsayisi.csv')
tufe = pd.read_csv('tufe.csv')
brent = pd.read_csv('Brent Petrol Vadeli İşlemleri Geçmiş Verileri.csv')
usd = pd.read_csv('USD_TRY Geçmiş Verileri.csv')
gini = pd.read_csv('gini_set.csv')


In [ ]:
df_train["Billing Date"] = pd.to_datetime(train["Billing Date"])
df_test["Week Start Date"] = pd.to_datetime(test["Week Start Date"])

price["Validfrom"] = pd.to_datetime(price["Validfrom"])
tufe["date"] = pd.to_datetime(tufe["date"])

usd["Tarih"] = pd.to_datetime(usd["Tarih"])
brent["Tarih"] = pd.to_datetime(brent["Tarih"])

df_train["year"] = df_train["Billing Date"].dt.year
df_train["month"] = df_train["Billing Date"].dt.month
df_train["week"] = df_train["Billing Date"].dt.weekofyear

df_test["year"] = df_test["Week Start Date"].dt.year
df_test["month"] = df_test["Week Start Date"].dt.month
df_test["week"] = df_test["Week Start Date"].dt.weekofyear

price["year"] = price["Validfrom"].dt.year
price["month"] = price["Validfrom"].dt.month
price["week"] = price["Validfrom"].dt.weekofyear

tufe["year"] = tufe["date"].dt.year
tufe["month"] = tufe["date"].dt.month


usd["year"] = usd["Tarih"].dt.year
usd["month"] = usd["Tarih"].dt.month
usd["week"] = usd["Tarih"].dt.weekofyear

brent["year"] = brent["Tarih"].dt.year
brent["month"] = brent["Tarih"].dt.month
brent["week"] = brent["Tarih"].dt.weekofyear

In [ ]:
price = price.groupby(["City Code","year","week","Product"])["CRP"].agg(["mean","std","max","min"]).reset_index()

price.columns = ["City Code","year","week","Product","CRP_mean","CRP_std","CRP_max","CRP_min"]

In [ ]:
usd.drop(["Açılış","Yüksek","Düşük","Hac.","Fark %","Düşük"],axis=1,inplace=True)

usd["Şimdi"] = usd["Şimdi"].apply(lambda x: x.replace(",",".")).astype(float)

brent["Şimdi"] = brent["Şimdi"].apply(lambda x: x.replace(",",".")).astype(float)

In [ ]:
usd = usd.groupby(["year","week"])["Şimdi"].agg(["mean","std","max","min"]).reset_index()

usd.columns = ["year","week","USD_mean","USD_std","USD_max","USD_min"]

brent = brent.groupby(["year","week"])["Şimdi"].agg(["mean","std","max","min"]).reset_index()

brent.columns = ["year","week","Brent_mean","Brent_std","Brent_max","Brent_min"]

In [ ]:
il_bölge = {"Antalya":"Akdeniz Bölgesi",
            "Burdur":"Akdeniz Bölgesi",
            "Isparta":"Akdeniz Bölgesi",
            "Mersin":"Akdeniz Bölgesi",
            "Adana":"Akdeniz Bölgesi",
            "Hatay":"Akdeniz Bölgesi",
            "Osmaniye":"Akdeniz Bölgesi",
            "K.Maraş":"Akdeniz Bölgesi",
            "İzmir":"Ege Bölgesi",
            "Aydın":"Ege Bölgesi",
            "Muğla":"Ege Bölgesi",
            "Manisa":"Ege Bölgesi",
            "Denizli":"Ege Bölgesi",
            "Uşak":"Ege Bölgesi",
            "Kütahya":"Ege Bölgesi",
            "Afyonkarahisar":"Ege Bölgesi",
            "Gaziantep":"Güneydoğu Anadolu Bölgesi",
            "Kilis":"Güneydoğu Anadolu Bölgesi",
            "Adıyaman":"Güneydoğu Anadolu Bölgesi",
            "Ş.Urfa":"Güneydoğu Anadolu Bölgesi",
            "Diyarbakır":"Güneydoğu Anadolu Bölgesi",
            "Mardin":"Güneydoğu Anadolu Bölgesi",
            "Batman":"Güneydoğu Anadolu Bölgesi",
            "Siirt":"Güneydoğu Anadolu Bölgesi",
            "Bolu":"Karadeniz Bölgesi",
            "Düzce":"Karadeniz Bölgesi",
            "Zonguldak":"Karadeniz Bölgesi",
            "Karabük":"Karadeniz Bölgesi",
            "Bartın":"Karadeniz Bölgesi",
            "Kastamonu":"Karadeniz Bölgesi",
            "Çorum":"Karadeniz Bölgesi",
            "Sinop":"Karadeniz Bölgesi",
            "Samsun":"Karadeniz Bölgesi",
            "Amasya":"Karadeniz Bölgesi",
            "Tokat":"Karadeniz Bölgesi",
            "Ordu":"Karadeniz Bölgesi",
            "Giresun":"Karadeniz Bölgesi",
            "Gümüşhane":"Karadeniz Bölgesi",
            "Trabzon":"Karadeniz Bölgesi",
            "Bayburt":"Karadeniz Bölgesi",
            "Rize":"Karadeniz Bölgesi",
            "Artvin":"Karadeniz Bölgesi",
            "Çanakkale":"Marmara Bölgesi",
            "Balıkesir":"Marmara Bölgesi",
            "Edirne":"Marmara Bölgesi",
            "Tekirdağ":"Marmara Bölgesi",
            "Kırklareli":"Marmara Bölgesi",
            "İstanbul":"Marmara Bölgesi",
            "Bursa":"Marmara Bölgesi",
            "Yalova":"Marmara Bölgesi",
            "Kocaeli":"Marmara Bölgesi",
            "Sakarya":"Marmara Bölgesi",
            "Bilecik":"Marmara Bölgesi",
            "Eskişehir":"İç Anadolu Bölgesi",
            "Konya":"İç Anadolu Bölgesi",
            "Ankara":"İç Anadolu Bölgesi",
            "Çankırı":"İç Anadolu Bölgesi",
            "Aksaray":"İç Anadolu Bölgesi",
            "Kırıkkale":"İç Anadolu Bölgesi",
            "Kırşehir":"İç Anadolu Bölgesi",
            "Yozgat":"İç Anadolu Bölgesi",
            "Niğde":"İç Anadolu Bölgesi",
            "Nevşehir":"İç Anadolu Bölgesi",
            "Kayseri":"İç Anadolu Bölgesi",
            "Karaman":"İç Anadolu Bölgesi",
            "Sivas":"İç Anadolu Bölgesi",
            "Malatya":"Doğu Anadolu Bölgesi",
            "Erzincan":"Doğu Anadolu Bölgesi",
            "Elazığ":"Doğu Anadolu Bölgesi",
            "Tunceli":"Doğu Anadolu Bölgesi",
            "Bingöl":"Doğu Anadolu Bölgesi",
            "Erzurum":"Doğu Anadolu Bölgesi",
            "Muş":"Doğu Anadolu Bölgesi",
            "Bitlis":"Doğu Anadolu Bölgesi",
            "Şırnak":"Doğu Anadolu Bölgesi",
            "Kars":"Doğu Anadolu Bölgesi",
            "Ağrı":"Doğu Anadolu Bölgesi",
            "Ardahan":"Doğu Anadolu Bölgesi",
            "Van":"Doğu Anadolu Bölgesi",
            "Iğdır":"Doğu Anadolu Bölgesi",
            "Hakkari":"Doğu Anadolu Bölgesi"}

In [ ]:
# giniye bölge ismini ekliyoruz
gini["bölge"] = gini["il"].map(il_bölge)